# YOLOv12/v13-Face Training & Evaluation

Version corrigée utilisant le fork local d'Ultralytics

## 1. 📦 Installation des dépendances

In [ ]:
# IMPORTANT: Ne PAS installer ultralytics via pip
# On utilise le fork local dans ./ultralytics/

# Installer seulement les dépendances (sans ultralytics)
!pip install -r requirements.txt -q

# Vérifier l'installation
import torch
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Setup paths et imports
import sys
from pathlib import Path
import os

# Ajouter le fork local au path
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('./widerface_evaluate'))

# Importer depuis le fork local
from ultralytics import YOLO
from ultralytics.utils import LOGGER, colorstr

print("✅ Fork local d'Ultralytics chargé avec succès")

## 2. 📥 Préparation du dataset WIDERFace

In [ ]:
# Télécharger et préparer le dataset WIDERFace

# Vérifier si le dataset existe déjà
widerface_config = Path('ultralytics/cfg/datasets/widerface.yaml')

if widerface_config.exists():
    print("✅ Configuration WIDERFace trouvée")
    
    # Lire la config pour voir le path des données
    import yaml
    with open(widerface_config, 'r') as f:
        config = yaml.safe_load(f)
    
    dataset_path = Path(config.get('path', 'datasets/widerface'))
    print(f"📁 Path dataset: {dataset_path}")
    
    if dataset_path.exists():
        print("✅ Dataset WIDERFace déjà présent")
    else:
        print("📥 Dataset non trouvé, préparation nécessaire...")
        if Path('scripts/prepare_widerface.py').exists():
            !python scripts/prepare_widerface.py
        else:
            print("⚠️ Script de préparation non trouvé. Téléchargement manuel nécessaire.")
else:
    print("❌ Configuration WIDERFace non trouvée")

## 3. 🏋️ Configuration de l'entraînement

In [ ]:
# Configuration commune pour l'entraînement
train_config = {
    'data': 'ultralytics/cfg/datasets/widerface.yaml',
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 4,
    'patience': 20,
    'project': 'runs/face',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'close_mosaic': 10,
    'amp': True
}

print("🔧 Configuration d'entraînement prête")
print(f"📊 Device: {train_config['device']}")
print(f"📐 Image size: {train_config['imgsz']}")
print(f"📦 Batch size: {train_config['batch']}")

## 4. 🚀 Entraînement YOLOv12-Face

In [ ]:
# Entraîner YOLOv12-Face (pas Enhanced)
print("🚀 Début de l'entraînement YOLOv12-Face...")

# Charger le modèle YOLOv12-Face
model_v12 = YOLO('ultralytics/cfg/models/v12/yolov12-face.yaml')

# Entraîner
results_v12 = model_v12.train(
    name='yolov12-face',
    **train_config
)

print("✅ Entraînement YOLOv12-Face terminé!")

## 5. 🚀 Entraînement YOLOv13-Face

In [ ]:
# Entraîner YOLOv13-Face
print("🚀 Début de l'entraînement YOLOv13-Face...")

# Charger le modèle YOLOv13-Face
model_v13 = YOLO('ultralytics/cfg/models/v13/yolov13-face.yaml')

# Entraîner
results_v13 = model_v13.train(
    name='yolov13-face',
    **train_config
)

print("✅ Entraînement YOLOv13-Face terminé!")

## 6. 📊 Évaluation WIDERFace avec widerface_evaluate

In [ ]:
# Fonction pour générer les prédictions au format WIDERFace
import cv2
import numpy as np
from tqdm import tqdm

def generate_widerface_predictions(model_path, output_dir, dataset_path):
    """Génère les prédictions au format WIDERFace"""
    
    model = YOLO(model_path)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Parcourir les images du dataset val
    val_images_dir = Path(dataset_path) / 'WIDER_val' / 'images'
    
    for event_dir in val_images_dir.iterdir():
        if event_dir.is_dir():
            event_output = output_path / event_dir.name
            event_output.mkdir(exist_ok=True)
            
            for img_path in event_dir.glob('*.jpg'):
                # Prédiction
                results = model(str(img_path), conf=0.001, iou=0.5)
                
                # Sauvegarder au format WIDERFace
                txt_name = img_path.stem + '.txt'
                txt_path = event_output / txt_name
                
                with open(txt_path, 'w') as f:
                    f.write(f"{img_path.stem}\n")
                    
                    if results[0].boxes is not None:
                        boxes = results[0].boxes
                        f.write(f"{len(boxes)}\n")
                        
                        for box in boxes.data:
                            x1, y1, x2, y2, conf, cls = box.cpu().numpy()
                            w = x2 - x1
                            h = y2 - y1
                            f.write(f"{x1:.1f} {y1:.1f} {w:.1f} {h:.1f} {conf:.3f}\n")
                    else:
                        f.write("0\n")
    
    print(f"✅ Prédictions sauvegardées dans {output_path}")

# Générer les prédictions pour les deux modèles
# generate_widerface_predictions(
#     'runs/face/yolov12-face/weights/best.pt',
#     'widerface_eval/yolov12_predictions',
#     'datasets/widerface'
# )

# generate_widerface_predictions(
#     'runs/face/yolov13-face/weights/best.pt',
#     'widerface_eval/yolov13_predictions',
#     'datasets/widerface'
# )

In [ ]:
# Évaluation avec widerface_evaluate
from evaluation import evaluation

def evaluate_widerface(pred_dir, model_name):
    """Lance l'évaluation WIDERFace officielle"""
    
    print(f"\n📊 Évaluation WIDERFace pour {model_name}...")
    
    # Path vers le fichier ground truth
    gt_path = 'datasets/widerface/wider_face_split/wider_face_val.mat'
    
    # Lancer l'évaluation
    evaluation(pred_dir, gt_path)
    
    # Lire les résultats
    result_files = [
        'wider_pr_info_bbox_face_event_val.txt',
        'wider_pr_curve_facebox_event_val.mat'
    ]
    
    # Parser les résultats Easy/Medium/Hard
    pr_info_file = Path(pred_dir).parent / result_files[0]
    if pr_info_file.exists():
        with open(pr_info_file, 'r') as f:
            lines = f.readlines()
            # Extraire les AP pour Easy/Medium/Hard
            # Format attendu dans le fichier
            print(f"\n📈 Résultats {model_name}:")
            for line in lines[-3:]:
                print(line.strip())

# Évaluer les modèles
# evaluate_widerface('widerface_eval/yolov12_predictions', 'YOLOv12-Face')
# evaluate_widerface('widerface_eval/yolov13_predictions', 'YOLOv13-Face')

## 7. 🔍 Test sur images

In [ ]:
# Test sur quelques images
import matplotlib.pyplot as plt

def test_and_visualize(model_path, test_image, model_name):
    """Test et visualise les résultats"""
    model = YOLO(model_path)
    
    # Prédiction
    results = model(test_image, conf=0.25)
    
    # Visualisation
    plt.figure(figsize=(10, 8))
    plt.imshow(results[0].plot())
    plt.title(f'{model_name} - Détections')
    plt.axis('off')
    plt.show()
    
    # Stats
    if results[0].boxes is not None:
        print(f"✅ {model_name}: {len(results[0].boxes)} visages détectés")
        print(f"   Confiances: {results[0].boxes.conf.cpu().numpy()}")
    else:
        print(f"❌ {model_name}: Aucun visage détecté")

# Tester sur une image exemple
# test_image = 'path/to/test/image.jpg'
# test_and_visualize('runs/face/yolov12-face/weights/best.pt', test_image, 'YOLOv12-Face')
# test_and_visualize('runs/face/yolov13-face/weights/best.pt', test_image, 'YOLOv13-Face')

## 8. 📈 Comparaison finale

In [ ]:
# Tableau de comparaison et visualisation
import pandas as pd
import matplotlib.pyplot as plt

print("\n📊 COMPARAISON FINALE YOLOv12 vs YOLOv13-Face\n" + "="*60)

# Données de comparaison (à remplacer par vos résultats réels)
comparison_data = {
    'Modèle': ['YOLOv5s-Face', 'YOLOv12-Face', 'YOLOv13-Face'],
    'Easy (%)': [94.67, 96.20, 97.20],
    'Medium (%)': [92.75, 94.80, 95.90],
    'Hard (%)': [83.03, 88.40, 91.30],
    'Params (M)': [7.1, 11.4, 15.7],
    'FLOPs (G)': [5.8, 28.6, 35.4],
    'FPS': [142, 142, 128]
}

df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

# Graphique de comparaison
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Performance par difficulté
models = comparison_data['Modèle']
x = np.arange(len(models))
width = 0.25

ax[0].bar(x - width, comparison_data['Easy (%)'], width, label='Easy', alpha=0.8)
ax[0].bar(x, comparison_data['Medium (%)'], width, label='Medium', alpha=0.8)
ax[0].bar(x + width, comparison_data['Hard (%)'], width, label='Hard', alpha=0.8)

ax[0].set_ylabel('AP (%)')
ax[0].set_title('Performance WIDERFace par Difficulté')
ax[0].set_xticks(x)
ax[0].set_xticklabels(models)
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Trade-off Params vs Performance
ax[1].scatter(comparison_data['Params (M)'], comparison_data['Hard (%)'], s=200, alpha=0.7)
for i, model in enumerate(models):
    ax[1].annotate(model, (comparison_data['Params (M)'][i], comparison_data['Hard (%)'][i]),
                   xytext=(5, 5), textcoords='offset points')

ax[1].set_xlabel('Paramètres (M)')
ax[1].set_ylabel('AP Hard (%)')
ax[1].set_title('Trade-off: Complexité vs Performance')
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('yolov12_v13_comparison.png', dpi=300)
plt.show()

print("\n🎯 Recommandations:")
print("- YOLOv12-Face: Équilibre optimal vitesse/précision pour applications temps réel")
print("- YOLOv13-Face: Précision maximale (+3% Hard) pour applications critiques")
print("- Amélioration significative vs YOLOv5: +5.4% (v12) et +8.3% (v13) sur Hard")